Вопросов, пока особо нет

# Отток клиентов телеком компании

Оператор связи «Ниединогоразрыва.ком» хочет научиться прогнозировать отток клиентов. Если выяснится, что пользователь планирует уйти, ему будут предложены промокоды и специальные условия. Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и договорах.

### Описание услуг

Оператор предоставляет два основных типа услуг: 

1. Стационарную телефонную связь. Возможно подключение телефонного аппарата к нескольким линиям одновременно.
2. Интернет. Подключение может быть двух типов: через телефонную линию (DSL, от англ. digital subscriber line, «цифровая абонентская линия») или оптоволоконный кабель (Fiber optic).  

Также доступны такие услуги:
- Интернет-безопасность: антивирус (DeviceProtection) и блокировка небезопасных сайтов (OnlineSecurity);
- Выделенная линия технической поддержки (TechSupport);
- Облачное хранилище файлов для резервного копирования данных (OnlineBackup);
- Стриминговое телевидение (StreamingTV) и каталог фильмов (StreamingMovies).

За услуги клиенты могут платить каждый месяц или заключить договор на 1–2 года. Доступны различные способы расчёта и возможность получения электронного чека.

###  Описание данных

Данные состоят из файлов, полученных из разных источников:
* `contract.csv` — информация о договоре;
* `personal.csv` — персональные данные клиента;
* `internet.csv` — информация об интернет-услугах;
* `phone.csv` — информация об услугах телефонии.

В данных предоставлены следующие признаки:
+ BeginDate – дата начала пользования услугами,
+ EndDate – дата окончания пользования услугами,
+ Type – тип оплаты: ежемесячный, годовой и тд,
+ PaperlessBilling – безналичный расчет,
+ PaymentMethod – способ оплаты,
+ MonthlyCharges – ежемесячные траты на услуги,
+ TotalCharges – всего потрачено денег на услуги
+ Dependents – наличие иждивенцев
+ Senior Citizen – наличие пенсионного статуса по возрасту
+ Partner – наличие супруга(и)
+ MultipleLines – наличие возможности ведения параллельных линий во время
звонка


Во всех файлах столбец customerID содержит код клиента.
Информация о договорах актуальна на 1 февраля 2020.

### План работы
1. Загрузка и ознакомление с данными
    + загрузка необходимых библиотек
    + загрузка данных
    + ознакомление и обозначение проблем для этапа предобработки
2. Предварительная обработка данных
    + объединение данных
    + обработка пропусков
    + преобразование типов
    + добавление новых признаков
3. Исследовательский анализ
    + проверка распределений данных
    + проверка зависимостей целевой переменной
    + формирование и проверка гипотез
4. Подготовка и обучение модели
    + разбиение данных на тренировочную и тестовую выборки
    + создание несколько моделей (pipeline c CV)
    + обучение и определение лучшей модели
5. Проверка работы модели
    + проверка на тестовой выборке
    + анализ результатов работы модели
6. Вывод
    + вывод
    + параметры финальной модели

### Критерии оценки работы

Основная метрика: **AUC-ROC**\
Дополнительная метрика: **Accuracy**\
Критерии оценки: **AUC-ROC >= 0.85**


## Загрузка и ознакомление с данными

Загрузим необходимые библиотеки

In [1]:
import os
import re
from IPython.display import display

import matplotlib as plt
import numpy as np
import pandas as pd

Зафиксируем глобальные переменные

In [2]:
MY_DIR = 'data/'
YA_DIR = '/datasets/final_provider/'
RANDOM_STATE = 270223

Считаем данные

In [3]:
def search_and_var_naming(dir):
    """
    Search all *.csv files in dir and creat dic varname:filepath
    dir -- directory with data
    sep -- win/lin filepath sep
    """
    csv_files = {}
    for root, dirs, files in os.walk(dir):
        counter = 0
        for file in files:
            if file.endswith(".csv"):
                counter += 1
                csv_files[file.split('.')[0]] = dir + file
        if counter == 0:
            print('В папке отсутствуют файлы формата *.csv')
        else:
            print(f'Количество найденных файлов *.csv: {counter}')
    return csv_files

In [4]:
if os.path.exists(MY_DIR):
    print('Используется локальная папка')
    files_and_path = search_and_var_naming(MY_DIR)
elif os.path.exists(YA_DIR):
    print('Используется репозиторий Yandex')    
    files_and_path = search_and_var_naming(YA_DIR)
else:
    print('Папка с данными не найдена')

Используется локальная папка
Количество найденных файлов *.csv: 4


Преобразуем и присвоим найденным файлам имена в соответствии с их названием

In [5]:
for key in files_and_path:
    exec(f'{key} = pd.read_csv(files_and_path[key])')
    print(key)

contract
internet
personal
phone


In [6]:
files = [contract, internet, personal, phone]

Преобразуем имена колонок в snake_case

In [7]:
for file in files:
    new_columns_name=[]
    for column in file.columns:
        new_columns_name.append(re.sub('(?!^)([A-Z]+)', r'_\1', column).lower())
    file.columns = new_columns_name

### Последовательно изучим общую информацию о всех представленных данных

#### contract

In [8]:
def base_info(df:pd.DataFrame):
    """
    Display some base info about df
    head, tail, info, describe
    """
    print('Head')
    display(df.head())
    print('*'*120)
    print('Tail')
    display(df.tail())
    print('*'*120)   
    print('Info')
    df.info()
    print('*'*120)
    print('Describe')
    display(df.describe(include='all'))


In [9]:
base_info(contract)

Head


,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


************************************************************************************************************************
Tail


,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.5
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.9
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.6
7042,3186-AJIEK,2014-08-01,No,Two year,Yes,Bank transfer (automatic),105.65,6844.5


************************************************************************************************************************
Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7043 non-null   object 
 1   begin_date         7043 non-null   object 
 2   end_date           7043 non-null   object 
 3   type               7043 non-null   object 
 4   paperless_billing  7043 non-null   object 
 5   payment_method     7043 non-null   object 
 6   monthly_charges    7043 non-null   float64
 7   total_charges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB
************************************************************************************************************************
Describe


,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges
count,7043,7043,7043,7043,7043,7043,7043.000000,7043
unique,7043,77,5,3,2,4,NaN,6531
top,7590-VHVEG,2014-02-01,No,Month-to-month,Yes,Electronic check,NaN,
freq,1,366,5174,3875,4171,2365,NaN,11
mean,NaN,NaN,NaN,NaN,NaN,NaN,64.761692,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,30.090047,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,18.250000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,35.500000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,70.350000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,89.850000,NaN


Явные пропуски отсутствуют. 
+ `begin_date` - необходимо будет преобразовать в формат даты и при анализе проверить утверждение, чем более долгий срок клиент с компанией тем меньшая вероятность его ухода
+ `end_date` - целевой признак, если клиент ушел дата будет обозначена также необходимо преобразовать в формат даты, и преобразовать с `begin_date` в срок обслуживания
+ `total_charges` - проверим на пропуски и преобразуем в числовой
+ `monthly_charges` - тип данных ожидаемый, проверим если средний чек совпадает 'кол-во месяцев' / `total_charges` == `monthly_charges`, то клиент не увеличивал кол-во услуг, меньше -> отказывался, больше -> добавлял

Количество уникальных ID совпадает с количеством строк, следовательно полные дубликаты отсутствуют. Попробывать разбить преобразовать в хэш текстовую часть а цифровую посмотреть может в ней регион, офис и т.д, может быть корреляция с датой то удалим

In [10]:
contract.paperless_billing.unique()

array(['Yes', 'No'], dtype=object)

`paperless_billing` - уникальные значения да/нет

In [11]:
contract.payment_method.unique()

array(['Electronic check', 'Mailed check', 'Bank transfer (automatic)',
       'Credit card (automatic)'], dtype=object)

`payment_method` варианты платежей, проверим на зависимость с `paperless_billing` и на возможность объединения по методу платежа check/automatic

In [12]:
contract.type.unique()

array(['Month-to-month', 'One year', 'Two year'], dtype=object)

`type` - тип платежа, логично будет преположить что чем на более длительный срок клиент оплачивает услуги тем меньше вероятность его ухода, проверим при анализе данных

#### internet

In [13]:
base_info(internet)

Head


,customer_id,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


************************************************************************************************************************
Tail


,customer_id,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies
5512,6840-RESVB,DSL,Yes,No,Yes,Yes,Yes,Yes
5513,2234-XADUH,Fiber optic,No,Yes,Yes,No,Yes,Yes
5514,4801-JZAZL,DSL,Yes,No,No,No,No,No
5515,8361-LTMKD,Fiber optic,No,No,No,No,No,No
5516,3186-AJIEK,Fiber optic,Yes,No,Yes,Yes,Yes,Yes


************************************************************************************************************************
Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer_id        5517 non-null   object
 1   internet_service   5517 non-null   object
 2   online_security    5517 non-null   object
 3   online_backup      5517 non-null   object
 4   device_protection  5517 non-null   object
 5   tech_support       5517 non-null   object
 6   streaming_tv       5517 non-null   object
 7   streaming_movies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB
************************************************************************************************************************
Describe


,customer_id,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies
count,5517,5517,5517,5517,5517,5517,5517,5517
unique,5517,2,2,2,2,2,2,2
top,7590-VHVEG,Fiber optic,No,No,No,No,No,No
freq,1,3096,3498,3088,3095,3473,2810,2785


In [14]:
internet.internet_service.unique()

array(['DSL', 'Fiber optic'], dtype=object)

Значений меньше чем в `contract` - не все пользователи пользуются услугой интернет. В столбце internet_service два вида подключения. В остальных подключена ли услуга да/нет
Преобразование типов не требуется

Сделаем проверку для будущего объеденения

In [15]:
len(internet[~internet['customer_id'].isin(contract['customer_id'])])

0

#### personal

In [16]:
base_info(personal)

Head


,customer_id,gender,senior_citizen,partner,dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


************************************************************************************************************************
Tail


,customer_id,gender,senior_citizen,partner,dependents
7038,6840-RESVB,Male,0,Yes,Yes
7039,2234-XADUH,Female,0,Yes,Yes
7040,4801-JZAZL,Female,0,Yes,Yes
7041,8361-LTMKD,Male,1,Yes,No
7042,3186-AJIEK,Male,0,No,No


************************************************************************************************************************
Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     7043 non-null   object
 1   gender          7043 non-null   object
 2   senior_citizen  7043 non-null   int64 
 3   partner         7043 non-null   object
 4   dependents      7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB
************************************************************************************************************************
Describe


,customer_id,gender,senior_citizen,partner,dependents
count,7043,7043,7043.000000,7043,7043
unique,7043,2,NaN,2,2
top,7590-VHVEG,Male,NaN,No,No
freq,1,3555,NaN,3641,4933
mean,NaN,NaN,0.162147,NaN,NaN
std,NaN,NaN,0.368612,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN
25%,NaN,NaN,0.000000,NaN,NaN
50%,NaN,NaN,0.000000,NaN,NaN
75%,NaN,NaN,0.000000,NaN,NaN


In [17]:
personal.gender.unique()

array(['Female', 'Male'], dtype=object)

In [18]:
personal.senior_citizen.unique()

array([0, 1], dtype=int64)

Значения в остальных столбцах да/нет

Проверка

In [19]:
len(personal[~personal['customer_id'].isin(contract['customer_id'])])


0

#### phone

In [20]:
base_info(phone)

Head


,customer_id,multiple_lines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


************************************************************************************************************************
Tail


,customer_id,multiple_lines
6356,2569-WGERO,No
6357,6840-RESVB,Yes
6358,2234-XADUH,Yes
6359,8361-LTMKD,Yes
6360,3186-AJIEK,No


************************************************************************************************************************
Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     6361 non-null   object
 1   multiple_lines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB
************************************************************************************************************************
Describe


,customer_id,multiple_lines
count,6361,6361
unique,6361,2
top,5575-GNVDE,No
freq,1,3390


phone - содержит информацию о пользователях телефоном и подключена ли услуга нескольких линий

In [21]:
len(phone[~phone['customer_id'].isin(contract['customer_id'])])

0

### Вывод:


В данном разделе мы провели загрузку и предварительное ознакомление с данными. Данные представлены в виде 4х файлов формата *.csv. 
Переименовали столбцы в более удобный формат для работы. Датафреймам присвоили имена соответственно именам файлов из которых загружали

`contract` Данные предоставлены с 2014-02-01 по . 7043 строки  по 8 признаков. Пропуски и дубликаты не найдены. Обнаружены несоответствия типов данных. Из двух столбцов даты начала контракта и его окончания необходимо будет сформировать длительность пользования услугами, и выделить целевую переменную - факт окончания контракта.

`internet` 5517 клиентов с 8-ю признаками. Преобразования не требуются.

`personal` 7043 строки с 5-ю признаками. Признак `senior_citizen` кодирован 1/0 вместо обычного yes/no

`phone` 6361 строка с 2-я признаками

Во всех таблица первичным ключом является `customer_id` по нему будем производить объединение таблиц.

## Предварительная обработка данных

Сведем все данные в единый датафрейм

Перед сведение добавим индикаторные столбцы наличия услуг интернет и телефона

In [22]:
internet['internet_service'] = 'Yes'
phone['phone_service'] = 'Yes'

In [23]:
df = contract.merge(personal, how='left', on='customer_id')
df = df.merge(internet, how='left', on='customer_id')
df = df.merge(phone, how='left', on='customer_id')

In [24]:
pd.DataFrame({'NaNs, %': round((df.isna().mean()*100),2), 
              'NaNs, qty': df.isna().sum()}).style.format('{:.2f}').background_gradient('coolwarm')

,"NaNs, %","NaNs, qty"
customer_id,0.00,0.00
begin_date,0.00,0.00
end_date,0.00,0.00
type,0.00,0.00
paperless_billing,0.00,0.00
payment_method,0.00,0.00
monthly_charges,0.00,0.00
total_charges,0.00,0.00
gender,0.00,0.00
senior_citizen,0.00,0.00


In [25]:
df = df.fillna('No')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        7043 non-null   object        
 1   begin_date         7043 non-null   datetime64[ns]
 2   end_date           7043 non-null   object        
 3   type               7043 non-null   object        
 4   paperless_billing  7043 non-null   object        
 5   payment_method     7043 non-null   object        
 6   monthly_charges    7043 non-null   float64       
 7   total_charges      7043 non-null   object        
 8   gender             7043 non-null   object        
 9   senior_citizen     7043 non-null   int64         
 10  partner            7043 non-null   object        
 11  dependents         7043 non-null   object        
 12  internet_service   7043 non-null   object        
 13  online_security    7043 non-null   object        
 14  online_b

Преобрзуем в формат даты времени

In [29]:
df['begin_date'] = pd.to_datetime(df['begin_date'])

При предварительном исследовании нам попадались даты с днем равным 1 проверим, если так следовательно данные представлены с разбивкой по месяцам

In [38]:
df['begin_date'].dt.day.unique()

array([1], dtype=int64)

In [40]:
df['end_date'].unique()

array(['No', '2019-12-01 00:00:00', '2019-11-01 00:00:00',
       '2019-10-01 00:00:00', '2020-01-01 00:00:00'], dtype=object)

Клиенты начали уходить с 2019-10-01. Странно что до этого за такой долгий срок работы из компании не ушел ни один клиент

In [41]:
df['target'] = np.where((df['end_date'] != 'No'), 0, 1)

In [43]:
df['target'].mean()

0.7346301292063041

Теперь добавим столбец с длительностью контракта до момента расторжения или если клиент не ушел на дату выгрузки

In [76]:
def term(end, begin):
    if end != 'No':
        return (pd.to_datetime(end) - begin)
    else:
        return (pd.Timestamp('2020-02-01') - begin)

In [115]:
df.term = round(((df.apply(lambda df: term(df['end_date'], df['begin_date']), axis=1).dt.days) / 30), 1)

In [119]:
df.head()

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,gender,senior_citizen,...,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,multiple_lines,phone_service,target
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,Female,0,...,Yes,No,Yes,No,No,No,No,No,No,1
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,Male,0,...,Yes,Yes,No,Yes,No,No,No,No,Yes,1
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,...,Yes,Yes,Yes,No,No,No,No,No,Yes,0
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,Male,0,...,Yes,Yes,No,Yes,Yes,No,No,No,No,1
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,...,Yes,No,No,No,No,No,No,No,Yes,0


In [121]:
.

0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: total_charges, Length: 7043, dtype: object